In [ ]:
# default_exp samplers.common

# Samplers Common

> Collection of helper functions and constants common to both Circuit and Protocol samplers

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
#export
import qsam.math as math
from qsam.circuit import partition

import itertools as it
import functools
import numpy as np

# Constants

In [ ]:
#export
ONE_QUBIT_GATES = {'H', 'X', 'Z'}
TWO_QUBIT_GATES = {'CNOT'}

GATE_GROUPS = {'p': ONE_QUBIT_GATES | TWO_QUBIT_GATES,
               'p1': ONE_QUBIT_GATES,
               'p2': TWO_QUBIT_GATES,
               }

# Data structures

In [ ]:
#export
from anytree import RenderTree, NodeMixin
from anytree.exporter import JsonExporter
from anytree.importer import JsonImporter, DictImporter

In [ ]:
#export
class CountNode(NodeMixin):
    def __init__(self, name, parent=None, children=None, counts=0, 
                 ckey=None, is_deterministic=False, is_fail=False):
        super(CountNode, self).__init__()
        
        self.name = name
        self.ckey = ckey
        self.counts = int(counts)
        self.is_deterministic = is_deterministic
        self.is_fail = is_fail
        
        self.parent = parent
        if children:
            self.children = children
            
    @property
    def rate(self):
        assert not self.ckey and not self.is_root
        if self.parent.counts < 2: return 0.5
        else: return self.counts / self.parent.counts

    @property
    def var(self, var_fn=math.Wilson_var):
        assert not self.ckey and not self.is_root
        if self.parent.counts < 2: return 1.0
        else: return var_fn(self.rate, self.parent.counts)
    
    def __str__(self):
        if self.is_root or self.ckey:
            return f'{self.name} ({self.counts})'
        else:
            return f'{self.name} ({self.counts}/{self.parent.counts}, {self.var:.2e})'

In [ ]:
#export
class SampleTree:
    def __init__(self, root=None):
        self.root = root
    
    def add(self, name, parent=None, **kwargs):
        if parent == None:
            if self.root == None:
                self.root = CountNode(name, **kwargs)
            return self.root
        else:
            child_match = [n for n in parent.children if n.name==name]
            if child_match: return child_match[0]
            else: return CountNode(name, parent=parent, **kwargs)
        
    def update(self, name, parent=None, **kwargs):
        node = self.add(name, parent, **kwargs)
        node.counts += 1
        return node
        
    def detach(self, node):
        node.parent = None
        
    def rate(self, const):
        p = 0
        for leaf in self.root.leaves:
            if not leaf.is_root and not leaf.parent.is_deterministic and leaf.is_fail: 
                prod = 1
                for n in leaf.path[1:]:
                    if n.ckey: prod *= const[n.ckey[0]][n.ckey[1]]
                    else: prod *= n.rate
                p += prod
        return p
    
    def var(self, const):
        var = 0
        for node in [n for n in self.root.descendants if not n.ckey]:
            if not node.parent.is_deterministic:
                twig = 1.0
                for n in node.path[1:-1]:
                    if n.ckey: twig *= const[n.ckey[0]][n.ckey[1]]
                    else: twig *= n.rate
                                    
                subtree = 0
                for leaf in node.leaves:
                    if not leaf.is_fail: # CHECK.
                        continue
                    
                    prod = 1
                    for n in leaf.iter_path_reverse():
                        if n == node: 
                            break
                        elif n.ckey: prod *= const[n.ckey[0]][n.ckey[1]]
                        else: prod *= n.rate
                    
                    # if any([n.is_fail for n in leaf.siblings]): prod /= 2
                        
                    subtree += prod
                    
                var += node.var * twig**2 * subtree**2
        return var
    
    def delta(self, const):
        path_sum = 0.0
        for leaf in self.root.leaves:
            if not leaf.is_root:
                prod = 1.0
                for n in leaf.path[1:]:
                    if n.ckey: prod *= const[n.ckey[0]][n.ckey[1]]
                    else: prod *= n.rate
                path_sum += prod
        return 1.0 - path_sum
    
    def __str__(self):
        return '\n'.join([f'{pre}{node}' for pre, _, node in RenderTree(self.root)])
    
    def display(self, fname='temp.png'):
        
        from anytree.exporter import UniqueDotExporter
        from IPython.display import Image, display ## ! requirements: jupyter, graphviz
        
        def edgeattrfunc(node, child):
            weight = 10.0 * child.counts / self.root.counts
            weight = 5.0 if weight > 5.0 else weight
            return f'penwidth="{weight}"'

        def nodeattrfunc(node):
            node_form = ""
            if node.is_fail: 
                node_form = ', style=filled, fillcolor="#ff9999"'
            return f'label="{node.name}"' + node_form

        img = UniqueDotExporter(self.root, 
                          edgeattrfunc=edgeattrfunc,
                          nodeattrfunc=nodeattrfunc).to_picture(fname)
        display(Image(fname))
        return fname
    
    def save(self, fname):
        exporter = JsonExporter(indent=2, sort_keys=False, default=str)
        with open(fname, 'w') as f:
            exporter.write(self.root, f)

    def load(self, fname):
        dictimporter = DictImporter(CountNode)
        importer = JsonImporter(dictimporter)
        with open(fname, 'r') as f:
            self.root = importer.read(f)
        return self

_____
# Other functions

## Subset analytics

In [ ]:
#export
def circuit_partitions(circ, partition_names):
    return [partition(circ, GATE_GROUPS[name]) for name in partition_names]

In [ ]:
#export
def circuit_weight_vectors(w_max, w_exclude={}):
    w_exclude = set((w,) if isinstance(w, int) else w for w in w_exclude)
    w_maxs = [w_max] if isinstance(w_max, int) else w_max

    w_upto_w_maxs = [tuple(range(w_max+1)) for w_max in w_maxs]
    w_vecs = list(it.product( *w_upto_w_maxs ))
    filtered_w_vecs = [w for w in w_vecs if w not in w_exclude]
    return filtered_w_vecs

In [ ]:
#export
def circuit_subset_occurence(partitions, partition_w_vecs, p_phy_per_partition):
    """Return (weight)x(p_phys) (parition) subset occurance matrix transforming p_SS vector to p_L vector"""
    n_partition_elems = np.array([len(p) for p in partitions])
    Aws = np.array([math.binom(w_vec, n_partition_elems, p_phy_per_partition) for w_vec in partition_w_vecs])
    Aws = np.product(Aws, axis=-1) # mult Aws for multi-parameter, i.e. multi-partitions
    return Aws

In [ ]:
#export
def protocol_partitions(circuits_dict, partition_names):
    return {c_hash: circuit_partitions(circ, partition_names)
           for c_hash, circ in circuits_dict.items() if circ._noisy}

In [ ]:
#export
def protocol_weight_vectors(partition_dict):
    return {c_hash: circuit_weight_vectors([len(p) for p in partitions]) 
            for c_hash, partitions in partition_dict.items()}

In [ ]:
#export
def protocol_subset_occurence(partition_dict, w_vecs_dict, p_phys):
    return {c_hash: circuit_subset_occurence(partition_dict[c_hash], w_vecs, p_phys)
            for c_hash, w_vecs in w_vecs_dict.items()}

## Subset filtering and selection

In [ ]:
#export
def w_plus1_filter(sampler, w_ids, circuit_hash, **kwargs):
    n_Aws = np.ma.array(sampler.Aws_pmax[circuit_hash])
    n_Aws[w_ids,] = np.ma.masked # mask existing (already sampled) subsets
    w_ids.append(np.argmax(n_Aws)) # add next most important subset as possible candidate
    return w_ids

In [ ]:
#export
def random_sel(w_ids, **kwargs):
    return np.random.choice(w_ids)

In [ ]:
#export
def ERV_sel(self, sampler, w_ids, circuit_hash, tree_node, **kwargs):
    erv_deltas = []
    v_L = sampler.tree.var(sampler.Aws_pmax)
    delta = sampler.tree.delta(sampler.Aws_pmax)
    for idx in w_ids:
        w_vec = sampler.w_vecs[circuit_hash][idx]

        _tree_node = sampler.tree.add(w_vec, parent=tree_node, ckey=(circuit_hash, idx))
        __tree_node = sampler.tree.add('FAIL', parent=_tree_node, is_fail=True)
        _delta = sampler.tree.delta(sampler.Aws_pmax)
        _rate = __tree_node.rate

        _tree_node.counts += 1
        v_L_minus = sampler.tree.var(sampler.Aws_pmax)

        __tree_node.counts += 1
        v_L_plus = sampler.tree.var(sampler.Aws_pmax)

        _v_L = _rate * v_L_plus + (1 - _rate) * v_L_minus
        erv_delta = np.abs(v_L - _v_L) + (delta - _delta) 
        # erv_delta = v_L - _v_L + (delta - _delta) # Check with new def of var
        erv_deltas.append( erv_delta )

        # revert the change
        _tree_node.counts -= 1
        __tree_node.counts -= 1
        if _tree_node.counts == 0: sampler.tree.detach(_tree_node)
        if __tree_node.counts == 0: sampler.tree.detach(__tree_node)

    idx = np.argmax(erv_deltas)
    return w_ids[idx]